In [ ]:
import os
import sys
sys.path.append(os.path.dirname(os.path.realpath('__file__')) + '/../../../')
from acaisdk.file import File
from acaisdk.project import Project
from acaisdk.fileset import FileSet
from acaisdk.job import Job
from acaisdk.meta import *
from acaisdk.utils import utils
from acaisdk import credentials
# from acaisdk import automl

utils.DEBUG = True  # print debug messages
workspace = os.path.dirname(os.path.realpath('__file__'))  # get current directory

In [ ]:
# Setting up project
# 
# Either:
# 1) Provide your existing token for some project
# 2) Fill in information for new project creation (project_name, project_admin, project_user, optional [csp, budget])
# Do not change root_token!
token = ''
force = False
    
project_name = ''
project_admin = ''
project_user = ''
csp = 'AZURE' # AWS/AZURE/GCP/PRIVATE
budget = 10 # Default, I have no idea what other options there are

try:
    p
except NameError:
    pass
else:
    if not force:
        if token == '':
            print("User token {} already exists, saving to variable 'token'. If you want to enforce new project, set 'force=true'.".format(p['user_token']))
            token = p['user_token']
    else:
        print("User token {} already exists but forcing new project.".format(p['user_token']))


if token != '':
    print("Logging in with existing credentials.")
    credentials.login(token)
else:
    print("Creating new project.")
    if project_name == '' or project_admin == '' or project_user == '':
        raise ValueError("Some of the 'project_name', 'project_admin', 'project_user' not provided!")
    
    root_token = 'EmDlCTBF1ppONSciYVd03M9xkmF6hFqW' 
    p = Project.create_project(project_name, root_token, project_admin, csp=csp, budget=budget)
    p = Project.create_user(project_name, p['project_admin_token'], project_user)
    token = p['user_token']
    print("New user token {}, saved to variable 'token'".format(token))

In [ ]:
# Set env variable needed for Phoebe
USE_PHOEBE = False
if USE_PHOEBE:
    os.environ["CLUSTER"] = 'PHOEBE'

In [ ]:
# This command must live in its own cell because chaining with another command trunctuates its output
File.list_dir('/')

In [ ]:
outputs = os.path.join(workspace, 'outputs/')
if not os.path.isdir(outputs):
    os.mkdir(outputs)
inputs = os.path.join(workspace, 'inputs/')
if not os.path.isdir(inputs):
    os.mkdir(inputs)
inf = [f for f in os.listdir(inputs) if os.path.isfile(os.path.join(inputs, f))]
print(inf)

if inf == []:
    raise FileNotFoundError("Data is not extracted in the inputs directory! Extract the data NLP-GloVe/fake-and-real-news-dataset.zip NLP-GloVe/glove-twitter.zip to inputs directory (e.g. tar -xvf glove-twitter.zip -C inputs)!")

In [ ]:
# In case dataset is not downloaded at all = NLP-GloVe/fake-and-real-news-dataset.zip NLP-GloVe/glove-twitter.zip
# not present do the following:
#
# Install kaggle (pip3 or pip)
# pip3 install kaggle
#
# Download dataset to the workdir of Sentiment-IMDB
# kaggle datasets download clmentbisaillon/fake-and-real-news-dataset
# kaggle datasets download icw123/glove-twitter
#
# Untar downloaded dataset to inputs dir
# tar -xvf clmentbisaillon/fake-and-real-news-dataset.zip -C inputs
# tar -xvf icw123/glove-twitter -C inputs

In [ ]:
File.convert_to_file_mapping([inputs], '/nlp-inputs/') \
    .files_to_upload \
    .upload() \
    .as_new_file_set('nlpglove.inputs')

In [ ]:
File.list_dir('/nlp-inputs/')

In [ ]:
# Upload code
code = os.path.join(workspace, 'nlp-code.zip')
File.upload({code: 'nlp-code.zip'})

In [ ]:
File.list_dir('/nlp-code/')

In [ ]:
# Job setting was OOM with 3Gi, 10Gi
# I went ahead with LARGE resources, I don't know how much was actually used

job_setting = {
    "v_cpu": "30",
    "memory": "100Gi",
    "gpu": "0",
    "command": "mkdir -p ./nlp-output/ && (pip install -r requirements.txt) && (python3 nlp.py ./nlp-inputs/ ./nlp-output/ glove.twitter.27B.100d.txt)",
    "container_image": "python:3.10.9",
    'input_file_set': 'nlpglove.inputs',
    'output_path': './nlp-output/',
    'code': 'nlp-code.zip:7',
    'description': 'NLP using GloVe Embeddings',
    'name': 'nlp-zipcode-tf-more-mem'
}

j = Job().with_attributes(job_setting).run()

In [ ]:
j.check_job_status(3286)

In [ ]:
from acaisdk.job import Job, JobStatus
status = j.wait()
if status == JobStatus.FINISHED:
    output_file_set = j.output_file_set
    print("Job done. output file set id:", output_file_set)
else:
    print("Job went wrong:", status)

In [ ]:
File.list_dir('/nlp-output/')

In [ ]:
File.download({'/nlp-output/accuracy.txt': outputs})